In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.base import clone
from sklearn.preprocessing import LabelEncoder
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import copy
from importlib import import_module
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from collections import OrderedDict
import copy


In [ ]:
le = LabelEncoder()

In [ ]:
class RBM(nn.Module):

    def __init__(self, n_visible, n_hidden, lr=1e-3, epochs=10, batch_size=30, k=3, use_gpu=True, verbose=True):
        super(RBM, self).__init__()

        self.n_visible = n_visible
        self.n_hidden = n_hidden
        self.lr = torch.tensor(lr, dtype=torch.float32)
        self.epochs = epochs
        self.batch_size = int(batch_size)
        self.k = k
        self.use_gpu = use_gpu
        self.verbose = verbose

        # Set the device to GPU if available
        if torch.cuda.is_available() and use_gpu==True:
            dev = "cuda:0"
        else:
            dev = "cpu"
        self.device_ = torch.device(dev)

        # Initialise weights and biases
        std = 4 * np.sqrt(6. / (self.n_visible + self.n_hidden))
        self.W = torch.normal(mean=0, std=std, size=(self.n_visible, self.n_hidden))
        self.vb = torch.zeros(self.n_visible)
        self.hb = torch.zeros(self.n_hidden)

        self.W = self.W.to(self.device_)
        self.vb = self.vb.to(self.device_)
        self.hb = self.hb.to(self.device_)

    def v_to_h(self, v):
        h = torch.matmul(v,self.W)   # calculate the activations of hidden units
        h = torch.add(h, self.hb)    # add bias term to the activations
        h = torch.sigmoid(h)
        return h, torch.bernoulli(h) # return both the probabilities and binary samples of hidden layer units

    def h_to_v(self, h):
        v = torch.matmul(h,self.W.t())  # calculate the activations of visible units
        v = torch.add(v, self.vb)       # add bias term to the activations
        v = torch.sigmoid(v)
        return v, torch.bernoulli(v)    # return both the probabilities and binary samples of visible layer units

    def contrastive_divergence(self, v0):
        # initial activations of hidden units and hidden samples using the input data
        h0, hkact = self.v_to_h(v0)

        # perform gibbs sampling k times to get the final samples of hidden and visible units
        for i in range(self.k):
            vk, _ = self.h_to_v(hkact)
            hk, hkact = self.v_to_h(vk)

        # compute delta for the parameters using the input and the final samples of hidden and visible units
        dW = torch.mm(v0.t(), h0) - torch.mm(vk.t(), hk) # delta for W
        dvb = torch.sum((v0-vk), 0) # delta for visible unit biases
        dhb = torch.sum((h0-hk), 0) # delta for hidden unit biases

        # update the parameters using the computed deltas
        self.W += self.lr * dW
        self.vb += self.lr * dvb
        self.hb += self.lr * dhb

        # compute the reconstruction error between the input and the final reconstructed visible layer
        err = torch.mean(torch.sum((v0 - vk)**2, 0))
        return err

    def forward(self, X):
        return self.v_to_h(X)

    def train(self, dataset):
        loader = DataLoader(dataset, batch_size=self.batch_size)

        for ep in range(self.epochs):
            running_cost = 0.
            n_batchs = 0
            for i, (batch, _) in enumerate(loader):
                batch = batch.view(len(batch), self.n_visible)
                running_cost += self.contrastive_divergence(batch)
                n_batchs += 1

            if self.verbose:
                print(f'RBM - Epoch: {ep}, averaged cost = {running_cost/n_batchs}')
        return

class GBRBM(RBM):
    def h_to_v(self,h):

        v = torch.matmul(h ,self.W.t()) # calculate the activations of visible units
        v = torch.add(v, self.vb)       # add bias term to the activations

        # return both the probabilities and gaussian samples of visible layer units
        return v, v + torch.normal(mean=0, std=1, size=v.shape).to(self.device_)


In [ ]:
class DBN(nn.Module):
    def __init__(self, n_visible, n_hiddens, lr=1e-3, epochs=100, batch_size=50, k=3,
                 use_gpu=True, verbose=True):
        super(DBN,self).__init__()

        self.n_layers = len(n_hiddens)
        self.n_visible = n_visible
        self.n_hiddens = n_hiddens
        self.use_gpu = use_gpu
        self.verbose = verbose
        self.lr = torch.tensor(lr, dtype=torch.float32)
        self.epochs = epochs
        self.batch_size = int(batch_size)
        self.k = k

        self.rbm_layers_ = []
        for i in range(self.n_layers):
            if i == 0:
                n_in = n_visible
                rbm = GBRBM(
                    n_in,
                    n_hiddens[0],
                    lr=lr,
                    epochs=epochs,
                    batch_size=batch_size,
                    k=k,
                    use_gpu=use_gpu,
                    verbose=verbose
                )
            else:
                n_in = n_hiddens[i-1]
                rbm = RBM(n_in,
                    n_hiddens[i],
                    lr=lr,
                    epochs=epochs,
                    batch_size=batch_size,
                    k=k,
                    use_gpu=use_gpu,
                    verbose=verbose
                )
            self.rbm_layers_.append(rbm)

    def forward(self, X):
        h = torch.as_tensor(X, dtype=torch.float)
        for rbm in self.rbm_layers_:
            h = h.view((h.shape[0], -1)) # flatten
            p_h, h = rbm.v_to_h(h)
        return p_h, h

    def pre_train(self, X):
        y = torch.zeros(X.shape[0])

        # train RBMs layer by layer
        for rbm in self.rbm_layers_:
            dataset = TensorDataset(X, y)
            rbm.train(dataset)

            # forward to next rbm
            X = X.view((X.shape[0], -1)) # flatten
            _, X = rbm.forward(X)

    def to_autoencoder(self, loss='MSELoss', optimizer='RMSprop', lr=0.01,
                       epochs=50, batch_size=50, loss_kwargs={}, optimizer_kwargs=dict()):
        return AEDBN(self, loss=loss, optimizer=optimizer, lr=lr, epochs=epochs, batch_size=batch_size,
                     loss_kwargs=loss_kwargs, optimizer_kwargs=optimizer_kwargs, verbose=self.verbose)

class AEDBN(nn.Module):

    def __init__(self, dbn, loss='MSELoss', optimizer='RMSprop', lr=0.01, epochs=50, batch_size=50,
                 loss_kwargs={}, optimizer_kwargs=dict(), verbose=True):
        super(AEDBN,self).__init__()

        self.dbn = dbn
        self.loss = loss
        self.optimizer = optimizer
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.loss_kwargs = loss_kwargs
        self.optimizer_kwargs = optimizer_kwargs

        self.construct_autoencoder()


    def construct_autoencoder(self):
        # unroll as an anto encoder/decoder
        n_in = self.dbn.n_visible

        # encoder part
        modules = []
        for n_hidden, rbm in zip(self.dbn.n_hiddens, self.dbn.rbm_layers_):
            layer = nn.Linear(n_in, n_hidden)
            layer.weight, layer.bias = nn.Parameter(rbm.W.t()), nn.Parameter(rbm.hb)
            modules.append(layer)
            modules.append(nn.Sigmoid())
            n_in = n_hidden
        self.encoder_ = nn.Sequential(*modules)

        # decoder part
        modules = []
        for i, n_hidden in enumerate(reversed(self.dbn.n_hiddens)):
            if i > 0:
                layer = nn.Linear(n_in, n_hidden)
                layer.weight, layer.bias = nn.Parameter(self.dbn.rbm_layers_[-i].W), nn.Parameter(self.dbn.rbm_layers_[-i].vb)
                modules.append(layer)
                modules.append(nn.Sigmoid())
                n_in = n_hidden
        layer = nn.Linear(n_hidden, self.dbn.n_visible)
        layer.weight, layer.bias = nn.Parameter(self.dbn.rbm_layers_[0].W), nn.Parameter(self.dbn.rbm_layers_[0].vb)
        modules.append(layer) # final output layer
        self.decoder_ = nn.Sequential(*modules)

    def forward(self, X):
        X = torch.as_tensor(X, dtype=torch.float)
        enc = self.encoder_(X)
        return self.decoder_(enc)

    def fine_tune(self, X, y=None):
        X = torch.as_tensor(X, dtype=torch.float)
        if y is None:
            y = X.detach().clone()

        # create dataset and data loader
        dataset = TensorDataset(X, y)
        loader = DataLoader(dataset, batch_size=self.batch_size)

        # set loss function and optimizer
        loss_fn = getattr(import_module('torch.nn'), self.loss)(**self.loss_kwargs)
        optimizer = getattr(import_module('torch.optim'), self.optimizer)
        optimizer = optimizer(self.parameters(), lr=self.lr, **self.optimizer_kwargs)

        # switch to training mode
        self.train(True)

        # train the model
        for ep in range(self.epochs):
            running_loss = 0.

            for i, (batch, y_batch) in enumerate(loader):
                optimizer.zero_grad()
                outputs = self.forward(batch)
                loss = loss_fn(outputs, y_batch)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            avg_loss = running_loss / len(loader)

            if self.verbose:
                print(f'AEDBN - Epoch {ep}, loss_train={avg_loss}')

        # switch back to evaluation mode
        self.train(False)

    def to_clf(self, n_class=5, loss='CrossEntropyLoss', optimizer='RMSprop', lr=0.01,
               epochs=50, batch_size=50, loss_kwargs={}, optimizer_kwargs=dict()):
        return CDBN(copy.deepcopy(self.encoder_), self.dbn.n_hiddens[-1], n_class=n_class, loss=loss,
                    optimizer=optimizer, lr=lr, epochs=epochs, batch_size=batch_size,
                    loss_kwargs=loss_kwargs, optimizer_kwargs=optimizer_kwargs, verbose=self.verbose)

class CDBN(nn.Module):

    def __init__(self, encoder, encode_size, n_class=5, loss='CrossEntropyLoss', optimizer='RMSprop', lr=0.01,
                 epochs=50, batch_size=50, loss_kwargs={}, optimizer_kwargs=dict(), verbose=True):
        super(CDBN,self).__init__()

        self.encoder = encoder
        self.encode_size = encode_size
        self.n_class = n_class
        self.loss = loss
        self.optimizer = optimizer
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.loss_kwargs = loss_kwargs
        self.optimizer_kwargs = optimizer_kwargs

        self.output_layer_ = nn.Linear(self.encode_size, n_class)


    def forward(self, X):
        X = torch.as_tensor(X, dtype=torch.float)
        return self.output_layer_(self.encoder(X))

    def fine_tune(self, X, y):
        X = torch.as_tensor(X, dtype=torch.float)

        # Create a DataLoader object to efficiently load the data in batches
        dataset = TensorDataset(X, y)
        loader = DataLoader(dataset, batch_size=self.batch_size)

        # Import the loss function and optimizer
        loss_fn = getattr(import_module('torch.nn'), self.loss)(**self.loss_kwargs)
        optimizer = getattr(import_module('torch.optim'), self.optimizer)
        optimizer = optimizer(self.parameters(), lr=self.lr, **self.optimizer_kwargs)

        # Set the model to training mode
        self.train(True)

        # Iterate over the epochs
        for ep in range(self.epochs):
            running_loss = 0.

            for i, (batch, y_batch) in enumerate(loader):
                optimizer.zero_grad()
                outputs = self.forward(batch)
                loss = loss_fn(outputs, y_batch)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            avg_loss = running_loss / len(loader)

            if self.verbose:
                print(f'CDBN - Epoch {ep}, loss_train={avg_loss}')

        # Set the model to evaluation mode
        self.train(False)

    def predict(self, X):

        X = torch.as_tensor(X, dtype=torch.float)

        self.eval()
        outputs = self.forward(X)
        softmax = nn.LogSoftmax(dim=1)
        _, y_pred = torch.max(softmax(outputs), dim=1)
        return y_pred


In [ ]:
class DBNClassifier(BaseEstimator, ClassifierMixin):

    def __init__(self, n_hiddens=[500, 100, 20], k=3, loss_ae='MSELoss', loss_clf='CrossEntropyLoss',
                 optimizer_ae='RMSprop', optimizer_clf='RMSprop',
                 lr_rbm=1e-3, lr_ae=0.01, lr_clf=0.01,
                 epochs_rbm=100, epochs_ae=50, epochs_clf=50,
                 batch_size_rbm=50, batch_size_ae=30, batch_size_clf=50,
                 loss_ae_kwargs={}, loss_clf_kwargs={},
                 optimizer_ae_kwargs={}, optimizer_clf_kwargs={}, random_state=42,
                 use_gpu=True, verbose=True):

        self.n_layers = len(n_hiddens)
        self.n_hiddens = n_hiddens
        self.k = k
        self.loss_ae = loss_ae
        self.loss_clf = loss_clf
        self.optimizer_ae = optimizer_ae
        self.optimizer_clf = optimizer_clf
        self.lr_rbm = lr_rbm
        self.lr_ae = lr_ae
        self.lr_clf = lr_clf
        self.epochs_rbm = epochs_rbm
        self.epochs_ae = epochs_ae
        self.epochs_clf = epochs_clf
        self.batch_size_rbm = int(batch_size_rbm)
        self.batch_size_ae = int(batch_size_ae)
        self.batch_size_clf = int(batch_size_clf)
        self.use_gpu = use_gpu
        self.verbose = verbose
        self.loss_ae_kwargs = loss_ae_kwargs
        self.loss_clf_kwargs = loss_clf_kwargs
        self.optimizer_ae_kwargs = optimizer_ae_kwargs
        self.optimizer_clf_kwargs = optimizer_clf_kwargs
        self.random_state = random_state
        self.le_ = LabelEncoder()



        if torch.cuda.is_available() and use_gpu==True:
            dev = "cuda:0"
        else:
            dev = "cpu"
        self.device_ = torch.device(dev)

        if not self.random_state is None:
            torch.manual_seed(self.random_state)


    def fit(self, X, y):
        try:
          X, y = check_X_y(X, y)
          self.n_features_in_ = X.shape[1]
          self.n_classes_ = np.unique(y).shape[0]

          self.le_.fit(y)

          X = torch.as_tensor(X, dtype=torch.float).to(self.device_)
          y = torch.as_tensor(self.le_.transform(y), dtype=torch.int64).to(self.device_)

          # pretrain DBN
          dbn = DBN(
              self.n_features_in_,
              self.n_hiddens,
              lr=self.lr_rbm,
              epochs=self.epochs_rbm,
              batch_size=self.batch_size_rbm,
              k=self.k,
              use_gpu=self.use_gpu,
              verbose=self.verbose
          )
          dbn.pre_train(X)

          # unroll as autoencoder-decoder and fine tune
          self.aedbn_ = dbn.to_autoencoder(
              loss=self.loss_ae,
              optimizer=self.optimizer_ae,
              lr=self.lr_ae,
              epochs=self.epochs_ae,
              batch_size=self.batch_size_ae,
              loss_kwargs=self.loss_ae_kwargs,
              optimizer_kwargs=self.optimizer_ae_kwargs
          )
          self.aedbn_.to(self.device_)
          self.aedbn_.fine_tune(X)

          # use the trained encoder, add an output layer and perform a supervised fine-tune
          self.cdbn_ = self.aedbn_.to_clf(
              n_class=self.n_classes_,
              loss=self.loss_clf,
              optimizer=self.optimizer_clf,
              lr=self.lr_clf,
              epochs=self.epochs_clf,
              batch_size=self.batch_size_clf,
              loss_kwargs=self.loss_clf_kwargs,
              optimizer_kwargs=self.optimizer_clf_kwargs
          )
          self.cdbn_.to(self.device_)
          self.cdbn_.fine_tune(X, y)

          return self
        except Exception as e:
          print(f"An error occurred during fitting: {e}")
          raise

    def predict(self, X):

        check_is_fitted(self)

        # Input validation
        X = check_array(X)
        X = torch.as_tensor(X).to(self.device_)

        return self.le_.inverse_transform(self.cdbn_.predict(X).cpu().detach().numpy())

class SimpleDBNClassifier(DBNClassifier):
    def __init__(self, n_hiddens=..., lr_pre_train=1e-5, lr_fine_tune=0.01,
                 epochs_pre_train=10, epochs_fine_tune=5, batch_size=30, k=3,
                 random_state=42, use_gpu=True, verbose=True):
        self.lr_pre_train = lr_pre_train
        self.lr_fine_tune = lr_fine_tune
        self.epochs_pre_train = epochs_pre_train
        self.epochs_fine_tune = epochs_fine_tune
        self.batch_size = batch_size

        super().__init__(n_hiddens, k=k, loss_ae='MSELoss', loss_clf='CrossEntropyLoss',
                         optimizer_ae='RMSprop', optimizer_clf='RMSprop',
                         lr_rbm=lr_pre_train, lr_ae=lr_fine_tune, lr_clf=lr_fine_tune,
                         epochs_rbm=epochs_pre_train, epochs_ae=epochs_fine_tune, epochs_clf=epochs_fine_tune,
                         batch_size_rbm=batch_size, batch_size_ae=batch_size, batch_size_clf=batch_size,
                         loss_ae_kwargs={}, loss_clf_kwargs={}, optimizer_ae_kwargs={},
                         optimizer_clf_kwargs={}, random_state=random_state,
                         use_gpu=use_gpu, verbose=verbose)


In [ ]:
X, y = load_digits(return_X_y=True)
print(X.shape)

(1797, 64)


In [ ]:
def local_train(client_id, clf, X_local, y_local):
    """
    Simulate local training on client with its own data.
    """
    clf.fit(X_local, y_local)  # Fit the model locally on each client’s data
    return clf

In [ ]:
# def federated_averaging(global_model, client_models):
#     # Average the parameters of the encoder
#     encoder_state_dict = OrderedDict()
#     for key in client_models[0].cdbn_.encoder.state_dict().keys():
#         encoder_state_dict[key] = sum(client_model.cdbn_.encoder.state_dict()[key] for client_model in client_models) / len(client_models)

#     # Average the parameters of the output layer
#     output_layer_state_dict = OrderedDict()
#     for key in client_models[0].cdbn_.output_layer_.state_dict().keys():
#         output_layer_state_dict[key] = sum(client_model.cdbn_.output_layer_.state_dict()[key] for client_model in client_models) / len(client_models)

#     # Update the global model
#     global_model.cdbn_ = copy.deepcopy(client_models[0].cdbn_)
#     global_model.cdbn_.encoder.load_state_dict(encoder_state_dict)
#     global_model.cdbn_.output_layer_.load_state_dict(output_layer_state_dict)

#     # Average other attributes if necessary
#     global_model.le_ = copy.deepcopy(client_models[0].le_)

#     return global_model

from collections import OrderedDict

def federated_averaging(global_model, client_models, client_data_sizes):
    # Calculate total data size across all clients
    total_data_size = sum(client_data_sizes)

    encoder_state_dict = OrderedDict()
    output_layer_state_dict = OrderedDict()

    # Weighted averaging of encoder parameters
    for key in client_models[0].cdbn_.encoder.state_dict().keys():
        encoder_state_dict[key] = sum(
            (client_data_size / total_data_size) * client_model.cdbn_.encoder.state_dict()[key]
            for client_model, client_data_size in zip(client_models, client_data_sizes)
        )

    # Weighted averaging of output layer parameters
    for key in client_models[0].cdbn_.output_layer_.state_dict().keys():
        output_layer_state_dict[key] = sum(
            (client_data_size / total_data_size) * client_model.cdbn_.output_layer_.state_dict()[key]
            for client_model, client_data_size in zip(client_models, client_data_sizes)
        )

    # Update the global model
    global_model.cdbn_ = copy.deepcopy(client_models[0].cdbn_)
    global_model.cdbn_.encoder.load_state_dict(encoder_state_dict)
    global_model.cdbn_.output_layer_.load_state_dict(output_layer_state_dict)

    return global_model


In [ ]:
def federated_training(clf, X, y, num_clients=3, rounds=10):
    X_splits = np.array_split(X, num_clients)
    y_splits = np.array_split(y, num_clients)
    client_data_sizes = [len(y_local) for y_local in y_splits]

    global_model = clone(clf)

    # Initialize the global model
    global_model.fit(X_splits[0], y_splits[0])

    for round in range(rounds):
        print(f"Round {round+1}/{rounds}")
        local_models = []

        for client_id in range(num_clients):
            print(f"Training on client {client_id+1}")
            local_model = clone(global_model)
            local_model.fit(X_splits[client_id], y_splits[client_id])
            local_models.append(local_model)

        global_model = federated_averaging(global_model, local_models, client_data_sizes)

    return global_model

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import ParameterGrid

# param_grid = {
#     'n_hiddens': [[500, 100, 20], [256, 128, 64], [1000, 500, 100]],
#     'lr_rbm': [1e-5, 1e-4, 1e-3],  # Reduced range
#     'lr_ae': [1e-4, 1e-3, 1e-2],   # Reduced range
#     'lr_clf': [1e-4, 1e-3, 1e-2],  # Reduced range
#     'epochs_rbm': [50, 100, 200],
#     'epochs_ae': [25, 50, 100],
#     'epochs_clf': [25, 50, 100],
#     'batch_size_rbm': [30, 50, 100],
#     'batch_size_ae': [30, 50, 100],
#     'batch_size_clf': [30, 50, 100],
#     'k': [1, 3, 5],
#     'num_clients': [3, 5, 7],
#     'rounds': [10, 15, 20]
# }

param_grid = {
    'n_hiddens': [256,128,64],
    'lr_rbm': 0.0001,
    'lr_ae': 0.001,
    'lr_clf': 0.01,
    'epochs_rbm': 50,
    'epochs_ae': 100,
    'epochs_clf': 100,
    'batch_size_rbm': 50,
    'batch_size_ae': 30,
    'batch_size_clf': 100,
    'k': 3,
    'num_clients': 3,
    'rounds': 15
}

# param_combinations = list(ParameterGrid(param_grid))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def train_evaluate_model(X, y, params):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    clf = DBNClassifier(
        n_hiddens=params['n_hiddens'],
        k=params['k'],
        lr_rbm=params['lr_rbm'],
        lr_ae=params['lr_ae'],
        lr_clf=params['lr_clf'],
        epochs_rbm=params['epochs_rbm'],
        epochs_ae=params['epochs_ae'],
        epochs_clf=params['epochs_clf'],
        batch_size_rbm=params['batch_size_rbm'],
        batch_size_ae=params['batch_size_ae'],
        batch_size_clf=params['batch_size_clf'],
        verbose=False
    )

    global_model = federated_training(
        clf, X_train, y_train,
        num_clients=params['num_clients'],
        rounds=params['rounds']
    )

    y_pred = global_model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)

    return accuracy, global_model

In [ ]:
# import numpy as np

# def hyperparameter_search(X, y, param_combinations, n_trials=10):
#     best_accuracy = 0
#     best_params = None
#     best_model = None

#     for i in range(n_trials):
#         params = np.random.choice(param_combinations)
#         accuracy, model = train_evaluate_model(X, y, params)

#         print(f"Trial {i+1}/{n_trials}: Accuracy = {accuracy}")
#         print(f"Parameters: {params}")

#         if accuracy > best_accuracy:
#             best_accuracy = accuracy
#             best_params = params
#             best_model = model

#     return best_accuracy, best_params, best_model

# # Perform the hyperparameter search
# best_accuracy, best_params, best_model = hyperparameter_search(X, y, param_combinations, n_trials=10)

# print(f"\nBest Accuracy: {best_accuracy}")
# print(f"Best Parameters: {best_params}")

In [ ]:
# Perform training and testing
accuracy, model = train_evaluate_model(X, y, param_grid)

# Display the results
print(f"Testing Accuracy: {accuracy}")
print(f"Trained Model: {model}")

Round 1/15
Training on client 1
Training on client 2
Training on client 3
Round 2/15
Training on client 1
Training on client 2
Training on client 3
Round 3/15
Training on client 1
Training on client 2
Training on client 3
Round 4/15
Training on client 1
Training on client 2
Training on client 3
Round 5/15
Training on client 1
Training on client 2
Training on client 3
Round 6/15
Training on client 1
Training on client 2
Training on client 3
Round 7/15
Training on client 1
Training on client 2
Training on client 3
Round 8/15
Training on client 1
Training on client 2
Training on client 3
Round 9/15
Training on client 1
Training on client 2
Training on client 3
Round 10/15
Training on client 1
Training on client 2
Training on client 3
Round 11/15
Training on client 1
Training on client 2
Training on client 3
Round 12/15
Training on client 1
Training on client 2
Training on client 3
Round 13/15
Training on client 1
Training on client 2
Training on client 3
Round 14/15
Training on client 1
T

In [ ]:
# clf = DBNClassifier(n_hiddens=[500, 100, 20], k=3,
#                     loss_ae='MSELoss', loss_clf='CrossEntropyLoss',
#                     optimizer_ae='RMSprop', optimizer_clf='RMSprop',
#                     lr_rbm=1e-4, lr_ae=0.001, lr_clf=0.001,
#                     epochs_rbm=100, epochs_ae=50, epochs_clf=50,
#                     batch_size_rbm=50, batch_size_ae=30, batch_size_clf=50,
#                     loss_ae_kwargs={}, loss_clf_kwargs={},
#                     optimizer_ae_kwargs=dict(), optimizer_clf_kwargs=dict(),
#                     random_state=42, use_gpu=True, verbose=False)
# # Federated learning simulation
# global_model = federated_training(clf, X_train, y_train, num_clients=3, rounds=15)
# print("Global Model is ready!!")


Round 1/15
Training on client 1
Training on client 2
Training on client 3
Round 2/15
Training on client 1
Training on client 2
Training on client 3
Round 3/15
Training on client 1
Training on client 2
Training on client 3
Round 4/15
Training on client 1
Training on client 2
Training on client 3
Round 5/15
Training on client 1
Training on client 2
Training on client 3
Round 6/15
Training on client 1
Training on client 2
Training on client 3


KeyboardInterrupt: 

In [ ]:
y_pred=model.predict(X_test)
print(y_pred)


[6 9 3 7 2 2 5 2 5 2 1 4 4 0 4 2 3 7 8 8 4 3 9 7 5 6 3 5 6 3 4 9 1 4 4 6 9
 4 7 6 6 9 1 3 6 1 3 0 6 5 5 1 9 5 6 0 9 0 0 1 0 4 5 2 4 5 7 0 7 5 9 9 5 4
 7 0 4 5 9 9 9 0 2 3 8 0 6 4 4 9 1 2 8 3 9 2 9 4 4 4 4 3 5 3 1 8 5 1 4 2 7
 7 4 4 1 9 2 7 8 7 2 6 9 4 2 7 2 7 5 8 7 5 7 7 0 6 6 4 2 8 0 9 4 6 9 9 6 9
 0 1 5 6 6 0 6 4 2 9 3 7 7 2 9 0 4 5 3 6 5 7 9 8 4 2 1 3 7 7 2 2 3 9 8 0 3
 2 2 5 6 9 9 4 1 5 4 2 3 6 4 8 5 9 5 7 1 9 4 8 1 5 4 4 9 6 1 2 6 0 4 5 2 7
 4 6 4 5 6 0 3 2 3 6 7 1 9 2 4 7 6 8 2 5 5 1 6 2 8 8 8 5 7 6 2 2 2 3 4 8 8
 3 6 0 3 7 7 0 1 0 4 5 1 5 3 6 0 4 1 0 0 3 6 5 9 7 3 5 9 9 9 8 5 3 3 2 0 5
 8 3 4 0 2 4 6 4 3 4 5 0 5 2 1 3 1 4 1 1 7 0 8 5 2 1 2 8 7 0 6 4 8 1 5 1 8
 4 5 8 7 9 8 6 0 4 2 0 7 9 2 9 5 2 7 7 9 9 7 4 3 7 3 5]


In [ ]:
test_score = model.score(X_test, y_test)
print(f"Test Score: {test_score}")

Test Score: 0.9166666666666666


In [ ]:
# try:
#     y_pred = global_model.predict(X_test)
#     print(y_pred)
# except Exception as e:
#     print(f"An error occurred during prediction: {e}")
#     import traceback
#     traceback.print_exc()

In [ ]:
# # Evaluate the global model on the test set
# test_score = global_model.score(X_test, y_test)
# print(f"Test Score: {test_score}")